In [1]:
###Importing packages we need###

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from matplotlib.cm import get_cmap
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
from matplotlib.lines import Line2D
from numpy import *
import xarray as xr
from netCDF4 import Dataset, num2date
import math
import pygrib
import glob
import matplotlib.patches as patches
import cdsapi
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import metpy as mp
import os
import warnings
warnings.filterwarnings("ignore")

print("Done importing modules")

Done importing modules


In [2]:
###Import the 3 CF tables and concatenate###

#Read in the 3 tables#
dfg = pd.read_csv('CF_good.csv')
dfmf = pd.read_csv('CF_mf_updated.csv')
dfu = pd.read_csv('CF_unclear_updated_new.csv')
                  
#Combine into 1 big file with all CF cases and reset index#
dfall = pd.concat([dfg, dfmf, dfu])
dfall.reset_index(drop=True, inplace=True)

#Convert 'date' column to datetime if it is not already#
dfall['date'] = pd.to_datetime(dfall['date'])

#Sort the DataFrame by 'date' and del duplicate#
dfall_sort = dfall.sort_values(by='date').reset_index(drop=True)
dfall_sort = dfall_sort.drop(260)
dfall_sort = dfall_sort.sort_values(by='date').reset_index(drop=True)

#Select the subsample for ts analysis#
#dfall_half = dfall_sort[0:362]
dfall_select = dfall_sort.copy()
dfall_select



,date,max_value,max_time
0,1950-01-04,12449.629923,1950-01-03 09:00:00
1,1950-01-30,10615.853132,1950-01-29 06:00:00
2,1950-02-22,1695.041284,1950-02-21 16:00:00
3,1950-03-01,2258.049045,1950-02-28 07:00:00
4,1950-03-08,2873.386661,1950-03-06 22:00:00
...,...,...,...
718,2022-09-22,5556.971814,2022-09-21 16:00:00
719,2022-11-11,907.817141,2022-11-10 03:00:00
720,2022-11-30,316.819875,2022-11-29 05:00:00
721,2022-12-14,1187.444094,2022-12-13 18:00:00


In [3]:
###Import a test case to see the climo files for anomaly computing###

climo_path = '/projects/zami4053/Synoptic_Research/Climatologies/climatology_1950-01-22 00:00:00.nc'
climo_test = xr.open_dataset(climo_path)
climo_test

<xarray.Dataset>
Dimensions:    (longitude: 41, latitude: 33)
Coordinates:
  * longitude  (longitude) float32 -110.0 -109.8 -109.5 ... -100.5 -100.2 -100.0
  * latitude   (latitude) float32 43.0 42.75 42.5 42.25 ... 35.5 35.25 35.0
Data variables:
    sp         (latitude, longitude) float32 ...
    t2m        (latitude, longitude) float32 ...
    d2m        (latitude, longitude) float32 ...

In [ ]:
# # Directory containing the NetCDF files
# data_dir = "/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_DATA/"

# # Sort the NetCDF files
# file_list = sorted(glob.glob(data_dir + "*.nc"))

# # Initialize an empty list to store DataArrays
# aligned_data_arrays = []
# insufficient_data_files = []

# # Extract dates from filenames
# file_dates = [pd.to_datetime('_'.join(file.split('_')[-3:]).split('.')[0], format="%m_%d_%Y") for file in file_list]

# # Add a new column to dfall with the formatted dates from the filenames
# dfall['file_date'] = pd.to_datetime(dfall['date'])

# # Sort both lists by date to ensure alignment
# dfall = dfall.sort_values('file_date').reset_index(drop=True)
# sorted_files_with_dates = sorted(zip(file_dates, file_list))

# # Loop through each row in dfall and find the corresponding file
# file_idx = 0

# for i, row in dfall[0:5].iterrows():
#     while file_idx < len(sorted_files_with_dates) and sorted_files_with_dates[file_idx][0] < row['file_date']:
#         file_idx += 1

#     if file_idx >= len(sorted_files_with_dates):
#         break

#     file_date, file = sorted_files_with_dates[file_idx]

#     if file_date != row['file_date']:
#         print(f"Skipping date {row['file_date']} due to no matching file")
#         continue

#     # Open each file
#     ds = xr.open_dataset(file)
#     selected_data = ds.sel(latitude=37.5, longitude=-103.0)['t2m']  # Select a grid cell and variable here
#     max_time = pd.to_datetime(row['max_time'])

#     # Calculate start and end times for the 25-hour period
#     hours_sel = 12
#     start_time = max_time - timedelta(hours=hours_sel)  # can adjust
#     end_time = max_time + timedelta(hours=hours_sel)    # can adjust
#     time_array = pd.date_range(start=start_time, end=end_time, freq='H')
#     print(time_array)

#     # Select the data for the 25-hour period
#     selected_data_t = selected_data.sel(time=slice(start_time, end_time))

#     # Only include the data if it has the full 25-hour period
#     if selected_data_t.shape[0] == hours_sel*2+1:
#         aligned_data_arrays.append(selected_data_t)
#     else:
#         print(f"Skipping file {file} due to insufficient data")
#         insufficient_data_files.append((file, selected_data_t.time.values))

# # Handle files with insufficient data
# for file, times in insufficient_data_files:
#     # Create an array of NaNs with the required shape
#     full_times = pd.date_range(start=start_time, end=end_time, freq='H')
#     nan_data = xr.DataArray(np.full((hours_sel*2+1,), np.nan), dims=['time'], coords={'time': full_times})
#     aligned_data_arrays.append(nan_data)

In [4]:
    ###Extract the 25 hour periods centered at t=0 and compute anomalies###

def centered_ts_anomalies(df):
      
    #Directory containing the NetCDF files and climo files#
    data_dir = "/pl/active/ATOC_SynopticMet/Michael/ERA5_CF_cases/CF_DATA/"
    climo_dir = "/projects/zami4053/Synoptic_Research/Climatologies/"

    #Sort the NetCDF files#
    file_list = sorted(glob.glob(data_dir + "*.nc"))

    #Initialize an empty list to store DataArrays#
    aligned_data_arrays = []
    insufficient_data_files = []

    #Extract dates from filenames and convert to datetime#
    file_dates = [pd.to_datetime('_'.join(file.split('_')[-3:]).split('.')[0], format="%m_%d_%Y") for file in file_list]
    dfall_select['file_date'] = pd.to_datetime(dfall_select['date'])

    #Sort both lists by date to ensure alignment#
    dfall = dfall_select.sort_values('file_date').reset_index(drop=True)
    sorted_files_with_dates = sorted(zip(file_dates, file_list))

    #Loop through each row in dfall and find the corresponding file#
    file_idx = 0

    for i, row in dfall.iterrows():  #can change from all to a few cases
        while file_idx < len(sorted_files_with_dates) and sorted_files_with_dates[file_idx][0] < row['file_date']:    
            file_idx += 1

        if file_idx >= len(sorted_files_with_dates):
            break

        file_date, file = sorted_files_with_dates[file_idx]

        if file_date != row['file_date']:
            print(f"Skipping date {row['file_date']} due to no matching file")
            continue

        #Open each file and sel gridpoint and var#
        ds = xr.open_dataset(file)
        selected_data = ds.sel(latitude=37.5, longitude=-103.0)['d2m']   #Select a grid cell and variable here
        max_time = pd.to_datetime(row['max_time'])

        #Calculate start and end times for the 25-hour period#
        hours_sel = 12
        start_time = max_time - timedelta(hours=hours_sel)   #can adjust
        end_time = max_time + timedelta(hours=hours_sel)     #can adjust
        time_array = pd.date_range(start=start_time, end=end_time, freq='H')
        #print(time_array)

        #Select the data for the 25-hour period#
        selected_data_t = selected_data.sel(time=slice(start_time, end_time))

        #Only include the data if it has the full 25-hour period#
        if selected_data_t.shape[0] == hours_sel*2+1:

            #Initialize an empty list to store anomalies#
            anomalies = []

            #Loop through each hour in the 25-hour period to construct anomalies#
            for time in time_array:

                #Construct the file path for the climatology file and handle edge cases#
                if time.month in [6, 7, 8]:
                    climo_file = os.path.join(climo_dir, "climatology_1950-09-01 00:00:00.nc")

                elif time.month == 2 and time.day == 29:
                    climo_file = os.path.join(climo_dir, "climatology_1950-03-01 00:00:00.nc")

                else:
                    climo_file = os.path.join(climo_dir, f"climatology_1950-{time.strftime('%m-%d')} 00:00:00.nc")

                if not os.path.exists(climo_file):
                    print(f"Climatology file not found: {climo_file}")
                    continue

                #Open the files and sel the same gridpoint and variable#
                climo_ds = xr.open_dataset(climo_file)
                climo_t2m = climo_ds.sel(latitude=37.5, longitude=-103.0)['d2m']  #can adjust this

                #Calculate the anomaly#
                anomaly = selected_data_t.sel(time=time) - climo_t2m
                anomalies.append(anomaly)

            #Combine anomalies into a single DataArray#
            anomalies_da = xr.concat(anomalies, dim='time')
            aligned_data_arrays.append(anomalies_da)

        else: 
            print(f"Skipping file {file} due to insufficient data")
            insufficient_data_files.append((file, selected_data_t.time.values))

    #Handle files with insufficient data and fill with Nans#
    for file, times in insufficient_data_files:

        #Create an array of NaNs with the required shape#
        full_times = pd.date_range(start=start_time, end=end_time, freq='H')
        nan_data = xr.DataArray(np.full((hours_sel*2+1,), np.nan), dims=['time'], coords={'time': full_times})

        #Append to the total array#
        aligned_data_arrays.append(nan_data)
        
        return aligned_data_arrays
    
aligned_data_arrays = centered_ts_anomalies(dfall_select)

IndentationError: expected an indented block after function definition on line 3 (2696610431.py, line 6)

In [ ]:
##Reshape and fix the xr###

#Combine all the aligned DataArrays along a new dimension#
aligned_combined_data = xr.concat(aligned_data_arrays, dim='time', coords='minimal')
case_N = int(len(aligned_combined_data['time']) / 25)

#Reshape the array#
reshaped_data = aligned_combined_data.values.reshape(case_N, 25)
reshaped_data

#Convert back to xr with proper dimensions and coordinates#
reshaped_data_array = xr.DataArray(reshaped_data, dims=['CF_events', 'time'], coords={'CF_events': np.arange(case_N), 'time': np.arange(25)})
reshaped_data_array

In [ ]:
###Make sure the timseries are working properly along with the composite###

#Plot all the time series#
plt.figure(figsize=(12, 8))

#Create the time lag array
time_values = np.arange(-12, 13, 1)  

for i in range(reshaped_data_array.shape[0]):
    
    plt.plot(time_values, reshaped_data_array[i, :], color='blue', alpha=0.1, linewidth=0.7)
    
plt.xlabel('Time evolution of CF Event [hrs]', fontsize=14)
plt.ylabel('2-m Dewpoint Anomalies [K]', fontsize=14)
plt.title('Timseries evolution of 2-m Dewpoint Anomalies', fontsize=18)

#Add a vertical line at t=0 (index 13)#
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='t=0')

#Calculate and plot the average line#
average_time_series = reshaped_data_array.mean(dim='CF_events')
plt.plot(time_values, average_time_series, color='black', linewidth=3, label='Composite Mean')
plt.legend()

#Draw vertical lines where the y-values are positive#
positive_indices = np.where(average_time_series > 0.5)[0]

#Add the sample size#
plt.annotate(f'N={case_N}', xy=(9.5, 12), xytext=(9.5, 12), fontsize=12)

if len(positive_indices) > 0:
    first_positive = time_values[positive_indices[0]]
    last_positive = time_values[positive_indices[-1]]
    
    #Draw vertical lines at the first and last positive y-values#
    plt.axvline(x=first_positive, color='orange', linestyle='-', linewidth=2)
    plt.axvline(x=last_positive, color='orange', linestyle='-', linewidth=2)

#Show all the x range#
plt.xticks(time_values)

plt.show()

In [ ]:
###Plot the quartiles instead of all the cases###

#Calculate the mean, 5th percentile, and 95th percentile#
average_time_series = reshaped_data_array.mean(dim='CF_events')
percentile_15 = reshaped_data_array.quantile(0.25, dim='CF_events')
percentile_85 = reshaped_data_array.quantile(0.75, dim='CF_events')

#Plot the mean line and shaded region#
plt.figure(figsize=(12, 8))

#Create the time lag array
time_values = np.arange(-12, 13, 1)  

#Plot the shaded region between the 5th and 95th percentiles#
plt.fill_between(time_values, percentile_15, percentile_85, color='blue', alpha=0.3, label='25th-75th Percentile Range')

#Plot the mean line#
plt.plot(time_values, average_time_series, color='black', linewidth=3, label='Composite Mean')

#Add a vertical line at t=0 (index 13)#
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='t=0')

#Add labels and titl3#
plt.xlabel('Time evolution of CF Event [hrs]', fontsize=14)
plt.ylabel('2-m Dewpoint Anomalies [K]', fontsize=14)
plt.title('Timeseries Evolution of Dewpoint Anomalies', fontsize=18)
plt.legend()

#Draw vertical lines where the y-values are positive#
positive_indices = np.where(average_time_series > 0.5)[0]

#Add the sample size#
plt.annotate(f'N={case_N}', xy=(9, 3.5), xytext=(9, 3.5), fontsize=12)

if len(positive_indices) > 0:
    first_positive = time_values[positive_indices[0]]
    last_positive = time_values[positive_indices[-1]]
    
    #Draw vertical lines at the first and last positive y-values#
    plt.axvline(x=first_positive, color='orange', linestyle='-', linewidth=2)
    plt.axvline(x=last_positive, color='orange', linestyle='-', linewidth=2)
    
#Show all the x range#
plt.xticks(time_values)

plt.show()